In [16]:
import pathlib
import os
from glob import glob
import numpy as np
import mne
from scipy.io import loadmat

subject_names = ["bm_014", 
                 "ca_001", 
                 "ca_019",
                 "cc_007",
                 "cm_013",
                 "co_006",
                 "dm_022",
                 "ds_021",
                 "el_018",
                 "gb_020",
                 "gh_017",
                 "gp_011",
                 "gv_005",
                 "lf_012",
                 "lr_008",
                 "mp_004",
                 "pe_009",
                 "pl_016",
                 "pr_015",
                 "ra_003",
                 "re_002",
                 "sg_010"] 

# Sort by increasing number id
sorted_subject_names = sorted(subject_names, key=lambda x: int(''.join(filter(str.isdigit, x))))

BASE_DIR = "/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING"

In [17]:
nruns = 4
ntrials = 20
nchannels = 349
nsamples = 9001

# Function to mark bad channels
def mark_bad_channels(mat_file, fif_file):
    flags = mat_file['ChannelFlag']
    bad_channels = [fif_file.ch_names[i] for i in range(fif_file.info['nchan']) if flags[i, 0] == -1]
    fif_file.info['bads'] = bad_channels
    return fif_file

for subject in sorted_subject_names:
    OUTPUT_DIR = f"{BASE_DIR}/Processed/osl_processing/Mot_Generation/{subject}"
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    ###############################################
    # Get the mne object template from the tSSS file
    ###############################################
    tSSS_pattern = f"{BASE_DIR}/Raw/tSSS/{subject}/*/*_R1_raw_tsss_mc_fix.fif"
    tSSS_PATH = glob(tSSS_pattern)
    tSSS_file = mne.io.read_raw_fif(tSSS_PATH[0])

    tSSS_file._data = np.zeros((nchannels, nsamples))  # Replace data with empty array

    ###############################################
    # Get the time series stored in the mat file
    ###############################################
    DATA_PATH = f"{BASE_DIR}/Processed/bst_database/Aging_Language/data/{subject}/Mot_Generation/"

    file_paths = []
    for trial_num in range(1, ntrials + 1):  # 20 trials per run
        file_name = f"data_Croix_Bsl_trial{trial_num:03d}.mat"
        file_path = os.path.join(DATA_PATH, file_name)
        if os.path.exists(file_path):  # Check if the file exists
            file_paths.append(file_path)

        for additional_trial_num in range(2, nruns + 1):  # 4 runs
            file_name = f"data_Croix_Bsl_trial{trial_num:03d}_{additional_trial_num:02d}.mat"
            file_path = os.path.join(DATA_PATH, file_name)
            if os.path.exists(file_path):  # Check if the file exists
                file_paths.append(file_path)

    for file_path in file_paths:  # Process the valid files
        mat_file = loadmat(file_path)
        fif_file = mne.io.RawArray(mat_file["F"], tSSS_file.info)

        # Mark bad channels
        fif_file = mark_bad_channels(mat_file, fif_file)

        ###############################################
        # Filtering & Resampling
        ###############################################
        fif_file_resampled = fif_file.copy().filter(
            l_freq=0.5, h_freq=125, method="iir",
            iir_params=dict(order=5, ftype="butter")
        ).resample(sfreq=250)

        # Clean file name and save
        fif_name_clean = os.path.basename(file_path).replace("data_Croix_Bsl_", '', 1).replace('.mat', '')
        fif_file_resampled.save(f"{OUTPUT_DIR}/{fif_name_clean}_raw.fif", overwrite=True)

        # Interpolate bad channels if any exist
        if fif_file.info['bads']:
            fif_preload = mne.io.read_raw_fif(f"{OUTPUT_DIR}/{fif_name_clean}_raw.fif", preload=True)
            fif_file_interp = fif_preload.interpolate_bads()
            fif_file_interp.save(f"{OUTPUT_DIR}/{fif_name_clean}_raw.fif", overwrite=True)

Opening raw data file /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Raw/tSSS/ca_001/200130/aging_ca_001_200130_run1_R1_raw_tsss_mc_fix.fif...


/tmp/ipykernel_3521176/1023877169.py:22: RuntimeWarning: This filename (/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Raw/tSSS/ca_001/200130/aging_ca_001_200130_run1_R1_raw_tsss_mc_fix.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  tSSS_file = mne.io.read_raw_fif(tSSS_PATH[0])


    Range : 277000 ... 582999 =    277.000 ...   582.999 secs
Ready.
Creating RawArray with float64 data, n_channels=349, n_times=9001
    Range : 0 ... 9000 =      0.000 ...     9.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 0.50, 125.00 Hz: -6.02, -6.02 dB

2 events found
Event IDs: [13 14]
2 events found
Event IDs: [13 14]
Overwriting existing file.
Writing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/ca_001/trial001_raw.fif
Closing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/ca_001/trial001_raw.fif
[done]
Creating RawArray with float64 d

/tmp/ipykernel_3521176/1023877169.py:22: RuntimeWarning: This filename (/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Raw/tSSS/re_002/200206/aging_re_002_200206_run1_R1_raw_tsss_mc_fix.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  tSSS_file = mne.io.read_raw_fif(tSSS_PATH[0])


    Range : 324000 ... 630999 =    324.000 ...   630.999 secs
Ready.
Creating RawArray with float64 data, n_channels=349, n_times=9001
    Range : 0 ... 9000 =      0.000 ...     9.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 0.50, 125.00 Hz: -6.02, -6.02 dB

2 events found
Event IDs: [13 14]
2 events found
Event IDs: [13 14]
Overwriting existing file.
Writing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/re_002/trial001_raw.fif
Closing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/re_002/trial001_raw.fif
[done]
Creating RawArray with float64 d

...

    Range : 100000 ... 409999 =    100.000 ...   409.999 secs
Ready.
Creating RawArray with float64 data, n_channels=349, n_times=9001
    Range : 0 ... 9000 =      0.000 ...     9.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 0.50, 125.00 Hz: -6.02, -6.02 dB

2 events found
Event IDs: [13 14]
2 events found
Event IDs: [13 14]
Overwriting existing file.
Writing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/gb_020/trial001_raw.fif
Closing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/gb_020/trial001_raw.fif
[done]
Opening raw data file /run/user/

/tmp/ipykernel_3521176/1023877169.py:22: RuntimeWarning: This filename (/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Raw/tSSS/ds_021/210602/aging_ds_021_210602_run1_R1_raw_tsss_mc_fix.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  tSSS_file = mne.io.read_raw_fif(tSSS_PATH[0])


    Range : 237000 ... 547999 =    237.000 ...   547.999 secs
Ready.
Creating RawArray with float64 data, n_channels=349, n_times=9001
    Range : 0 ... 9000 =      0.000 ...     9.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 0.50, 125.00 Hz: -6.02, -6.02 dB

2 events found
Event IDs: [13 14]
2 events found
Event IDs: [13 14]
Overwriting existing file.
Writing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/ds_021/trial001_raw.fif
Closing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/ds_021/trial001_raw.fif
[done]
Opening raw data file /run/user/

/tmp/ipykernel_3521176/1023877169.py:22: RuntimeWarning: This filename (/run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Raw/tSSS/dm_022/210602/aging_dm_022_210602_run1_R1_raw_tsss_mc_fix.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  tSSS_file = mne.io.read_raw_fif(tSSS_PATH[0])


    Range : 47000 ... 357999 =     47.000 ...   357.999 secs
Ready.
Creating RawArray with float64 data, n_channels=349, n_times=9001
    Range : 0 ... 9000 =      0.000 ...     9.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1.2e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 0.50, 125.00 Hz: -6.02, -6.02 dB

2 events found
Event IDs: [13 14]
2 events found
Event IDs: [13 14]
Overwriting existing file.
Writing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/dm_022/trial001_raw.fif
Closing /run/user/1001/gvfs/smb-share:server=sum-lpnc-nas.u-ga.fr,share=securevault/LPNC-SecureVault/MEGAGING/Processed/osl_processing/Mot_Generation/dm_022/trial001_raw.fif
[done]
Opening raw data file /run/user/1

Events

In [18]:
# sfreq = 250

# # Define sample points
# baseline_start_sample = int(0 * sfreq)  # Start of the baseline
# presentation_sample = int(2 * sfreq)    # Start of the presentation
# generation_sample = int(4 * sfreq)      # Start of the generation

# # Define event IDs
# event_id_baseline = 1
# event_id_presentation = 2
# event_id_generation = 3

# # Create the events array
# events = np.array([
#     [baseline_start_sample, 0, event_id_baseline],
#     [presentation_sample, 0, event_id_presentation],
#     [generation_sample, 0, event_id_generation]
# ])

# fif_file_processed.plot(events=events)